In [1]:
import vk_api
from math import ceil
import pandas as pd

In [2]:
def chunks(l, n):
    list = []
    for i in range(0, len(l), n):
        list.append(l[i:i+n])
    return list

In [3]:
vk_session = vk_api.VkApi(token='')

In [4]:
vk = vk_session.get_api()

In [19]:
# group_members список пользователей, состоящих в группе:
response = vk.groups.getMembers(group_id=108367462)
group_members_id = response['items']
len(group_members_id)

1000

In [25]:
tools = vk_api.VkTools(vk_session)
group_members_id = tools.get_all('groups.getMembers', 1000, {'group_id': 108367462})
group_members_id = group_members_id['items']
print(len(group_members_id))

3531


In [8]:
# Друзья каждого пользовтеля, состоящего в группе:
for user_id in group_members_id[:2]:
    dict_of_friends = vk.friends.get(user_id=user_id, order='hints', 
                         fields=['sex, bdate, city, country, education, universities, schools, relation'])
    dict_of_friends['user_id'] = user_id
    print(dict_of_friends)

{'user_id': 33186, 'count': 668, 'items': [{'last_name': 'Сазонова', 'id': 747, 'first_name': 'Алёна', 'city': {'id': 2, 'title': 'Санкт-Петербург'}, 'online': 0, 'bdate': '22.7.1989', 'sex': 1, 'country': {'id': 1, 'title': 'Россия'}}, {'last_name': 'Станков', 'id': 795, 'first_name': 'Сергей', 'city': {'id': 2, 'title': 'Санкт-Петербург'}, 'online': 0, 'bdate': '13.9.1989', 'sex': 2, 'country': {'id': 1, 'title': 'Россия'}}, {'last_name': 'Набегаев', 'id': 811, 'first_name': 'Антон', 'city': {'id': 2, 'title': 'Санкт-Петербург'}, 'online': 1, 'bdate': '26.8.1988', 'sex': 2, 'country': {'id': 1, 'title': 'Россия'}}, {'last_name': 'Громова', 'id': 976, 'first_name': 'Тома', 'city': {'id': 2, 'title': 'Санкт-Петербург'}, 'online': 0, 'bdate': '1.5', 'sex': 1, 'country': {'id': 1, 'title': 'Россия'}}, {'last_name': 'Маслак', 'id': 4413, 'first_name': 'Андрей', 'city': {'id': 2, 'title': 'Санкт-Петербург'}, 'online': 0, 'bdate': '31.12', 'sex': 2, 'country': {'id': 1, 'title': 'Россия'}},

In [9]:
# получаю много всякой инфы от пользователей в словаре

for user_id in group_members_id[:1]:
    user_id_friends = vk.friends.get(user_id=user_id, order='hints') # словарь друзей, с полями count и items
    if user_id_friends['count'] < 999:
        user_id_friends_str = ','.join(str(str_user_id_friends) for str_user_id_friends in user_id_friends['items'])
        user_id_friends['items'] = vk.users.get(user_ids=user_id_friends_str, name_case='nom', fields='sex,bdate,city,country,education,universities,schools,relation,groups,friends,personal,interests')
    else:
        response = []
        user_id_friends_list_of_lists = chunks(user_id_friends_list, 999)
        for thousand_of_user_id_friends_list in user_id_friends_list_of_lists:
            user_id_friends_str = ','.join(str(str_user_id_friends) for str_user_id_friends in thousand_of_user_id_friends_lis)
            response.extend(vk.users.get(user_ids=user_id_friends_str, 
                                         name_case='nom', fields='groups,friends,personal,interests'))
        user_id_friends['items'] = response
    user_id_friends['user_id'] = user_id   
    print(user_id_friends)

    

{'user_id': 33186, 'count': 668, 'items': [{'last_name': 'Сазонова', 'id': 747, 'first_name': 'Алёна', 'city': {'id': 2, 'title': 'Санкт-Петербург'}, 'bdate': '22.7.1989', 'sex': 1, 'country': {'id': 1, 'title': 'Россия'}}, {'last_name': 'Станков', 'id': 795, 'first_name': 'Сергей', 'city': {'id': 2, 'title': 'Санкт-Петербург'}, 'bdate': '13.9.1989', 'sex': 2, 'country': {'id': 1, 'title': 'Россия'}}, {'last_name': 'Набегаев', 'id': 811, 'first_name': 'Антон', 'city': {'id': 2, 'title': 'Санкт-Петербург'}, 'bdate': '26.8.1988', 'sex': 2, 'country': {'id': 1, 'title': 'Россия'}}, {'last_name': 'Громова', 'id': 976, 'first_name': 'Тома', 'city': {'id': 2, 'title': 'Санкт-Петербург'}, 'bdate': '1.5', 'sex': 1, 'country': {'id': 1, 'title': 'Россия'}}, {'last_name': 'Маслак', 'id': 4413, 'first_name': 'Андрей', 'city': {'id': 2, 'title': 'Санкт-Петербург'}, 'bdate': '31.12', 'sex': 2, 'country': {'id': 1, 'title': 'Россия'}}, {'last_name': 'Снигирь', 'id': 5878, 'first_name': 'Ирина', 'cit

In [10]:
# Получаю id member и id member friends
all_users_id_set = set()
for user_id in group_members_id:
    try:
        user_id_friends = vk.friends.get(user_id=user_id, order='hints') # словарь друзей, с полями count и items
    except Exception:
        continue
    all_users_id_set = all_users_id_set | set(user_id_friends['items'])
    
all_users_id_str = ','.join([str(group_id) for group_id in all_users_id_set])   
all_users_id_list = [user_id for user_id in all_users_id_set]
print('Всего членов сообщества и их друзей: ', len(all_users_id_list))
# print(all_users_id_str)

# groups.get

Всего членов сообщества и их друзей:  468472


In [11]:
data = pd.read_csv('data/vk/user_and_friends_info.csv')
print(data.shape)
print(group_members_id)

# как добавить столбец из мембер??????!!!!!!!!!

data[is_member] = data[[data[user_id] == group_members_id]]

# user_info = pd.DataFrame(data=all_users_id_list, columns='user_id', index=all_users_id_list)
# user_info['user_id'] == group_members_id
# # user_info['is_member'] = user_info[[]
# user_info

(32021, 12)
[33186, 36415, 37259, 63923, 71914, 160308, 202190, 225365, 235404, 236711, 256110, 269126, 280926, 281782, 304252, 369818, 382491, 417914, 423598, 455548, 467709, 478070, 506753, 518934, 531359, 569663, 571574, 594540, 604396, 609579, 648320, 652691, 657236, 673666, 676897, 699388, 724244, 730935, 732809, 742366, 752811, 758217, 772153, 793260, 801360, 812486, 827095, 831156, 832925, 862735, 873519, 884111, 891065, 898823, 906429, 916385, 922125, 943264, 954506, 957231, 972727, 1058399, 1079358, 1098727, 1178337, 1178413, 1188100, 1194074, 1211885, 1217963, 1253067, 1255352, 1263668, 1280727, 1288897, 1292512, 1293624, 1333362, 1348449, 1371101, 1468986, 1493119, 1519741, 1539154, 1551472, 1563194, 1567795, 1584886, 1585352, 1652319, 1680131, 1706364, 1714037, 1717379, 1725141, 1733595, 1765526, 1804505, 1810171, 1855109, 1873937, 1920530, 1927604, 1936710, 1965611, 1967652, 1983151, 2000984, 2042245, 2054816, 2087419, 2121893, 2137851, 2143205, 2227752, 2234594, 2254624, 

KeyError: 30096742

In [14]:
from tqdm import tqdm

In [26]:
# работа с группами:
mid_table = pd.DataFrame(columns=['user_id', 'group_id'])

print(len(group_members_id))

for user_id in tqdm(group_members_id):
    try:
        response = vk.groups.get(user_id=user_id)
    except Exception:
        continue  
    for group_id in response['items']:
        mid_table = mid_table.append({'user_id': user_id, 'group_id': group_id}, ignore_index=True)
        
mid_table.to_csv('data/vk/mit_table.csv')

mid_table.user_id.value_counts()
    


  0%|          | 0/3531 [00:00<?, ?it/s]

3531



  0%|          | 9/3531 [00:07<50:48,  1.16it/s]Exception in thread Thread-6:
Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/anaconda3/lib/python3.5/site-packages/tqdm/_tqdm.py", line 102, in run
    for instance in self.tqdm_cls._instances:
  File "/anaconda3/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 3531/3531 [1:21:26<00:00,  2.03it/s]


233722385.0    4987
360068399.0    4907
14677019.0     4750
330539934.0    4177
31358630.0     4098
148146067.0    4070
77783034.0     3827
249151419.0    3656
21029628.0     3507
160796070.0    3501
277403019.0    3256
146918701.0    3233
144592172.0    3178
21327613.0     3130
292035565.0    2996
97516657.0     2885
199608552.0    2880
347421513.0    2877
45306327.0     2853
183147745.0    2814
302552542.0    2802
171109061.0    2696
78195326.0     2600
326272305.0    2499
274955966.0    2356
4741912.0      2349
6086117.0      2330
58086559.0     2309
1680131.0      2248
189740725.0    2227
               ... 
87365969.0        7
238058664.0       7
399720460.0       7
341520482.0       7
339114445.0       7
182252576.0       6
337220930.0       6
342388532.0       6
238167293.0       6
337234496.0       5
364266156.0       5
188858910.0       5
344309632.0       5
343918997.0       5
248551631.0       5
30096742.0        5
59493617.0        5
62057151.0        5
95746207.0        5


In [59]:
# Получение последней таблиц
mid_table = pd.read_csv('data/vk/mit_table.csv')
all_group_id = mid_table.group_id.drop_duplicates().tolist() #Вернет все id групп
print(len(all_group_id))

desc_of_group = []

for five_hund in tqdm(chunks(all_group_id, 500)):
    five_hund_str = ','.join(str(int(group_id)) for group_id in five_hund) 
    
    for group in vk.groups.getById(group_ids=five_hund_str, fields='description'): 
        try:
            desc_of_group.append( (group['id'], group['description']) )
        except:
            continue
        #desc_of_group = desc_of_group.append({'group_id': group['id'], 'description': group['description']}, index=True)
    
# all_group_id_str = ','.join(str(group_id) for group_id in all_group_id)
               
    
desc_of_group = pd.DataFrame(desc_of_group, columns=['group_id', 'description'])

  0%|          | 0/654 [00:00<?, ?it/s]

326936


100%|██████████| 654/654 [18:42<00:00,  1.77s/it]


In [57]:
desc_of_group.shape

(986, 2)

In [50]:
five_hund_str


'88350989,37436948,44698558,94575049,87748076,74954668,67977655,73771577,95860056,108367462'

In [55]:
group

{'deactivated': 'banned',
 'id': 28816502,
 'is_closed': 0,
 'name': 'ZEVS.IN "МАГНИТ УСПЕХА"',
 'photo_100': 'https://vk.com/images/deactivated_100.png',
 'photo_200': 'https://vk.com/images/deactivated_200.png',
 'photo_50': 'https://vk.com/images/deactivated_50.png',
 'screen_name': 'shop_mb',
 'type': 'page'}

In [61]:
desc_of_group.to_csv('data/vk/desc_of_group.csv')